# Assignment-Adapter Demo
In this we look at another builtin adapter, the Simple Assignment Adapter `enbios/base/adapters_aggregators/builtin/assignment_adapter.py`. This adapter does not any specific calculations. Instead, it allows the user to introduce fixed values, that should come from some external source into the enbios tree calculation. This includes not just the outputs of structural nodes, but in particular their impact results.

These values can be either in the experiment configuration file or for convenience in a referenced csv file. The values can be specified in such a way, that scenario outputs and result values have consistent and valid units.

In addition, compared to the brightway adapter, this adapter allows to specify _multiple outputs_ for each node, and also nodes that do not specify any output, even tho they have some impacts. 

The adapters dedicated name is `assignment-adapter` tho for nodes, the indicator `assign` can also be used.

We will start with a minimal experiment configuration, that includes one node with a simple assignment 

In [1]:
from enbios import Experiment
from enbios.base.adapters_aggregators.builtin.assignment_adapter import AssignmentAdapter

# let's keep this node separate, so we can manipulate it easier later
simple_assignment_node = {
    "name": "simple_node",
    "adapter": "assign",
    # the config of an assign node needs either: default_impacts or scenario_data.impacts
    "config": {
        "outputs": [{"unit":"l"}],
        "default_outputs": [{"magnitude": 4}],
        "default_impacts": {"co2": {"unit": "kg", "magnitude": 10}},
    },
}

experiment_data = {
    "adapters": [
        {
            "adapter_name": "assignment-adapter",
            # by default this adapter does no specific config
            "methods": {"co2": "kg"},
        }
    ],
    "hierarchy": {
        "name": "root",
        "aggregator": "sum",
        "children": [simple_assignment_node],
    },
}

In [2]:
experiment = Experiment(experiment_data)
experiment.run()

2024-04-10 11:30:06,989 - ............demos.enbios.base - INFO - Running scenario 'default scenario'


{'default scenario': {'name': 'root',
  'results': {'co2': {'unit': 'kg', 'magnitude': 10.0}},
  'output': [{'unit': 'liter', 'magnitude': 4.0}],
  'children': [{'name': 'simple_node',
    'results': {'co2': {'unit': 'kg', 'magnitude': 10.0}},
    'output': [{'unit': 'l', 'magnitude': 4.0}]}]}}

The config of an assign node is structured the following way:
These are the fields that can/must be set:
```
    node_name: a unique string
    outputs: a list that specifies the outputs
    default_outputs: a list of default-outputs (optional) 
    default_impacts: a dictionary of default impacts
```

In detail:




In [12]:
experiment_data["scenarios"] = [
    {
        "name": "scenario1",
        "nodes": {
            "simple_node": {
                "outputs": [{"unit": "l", "magnitude": 100}],
                "impacts": {"co2": {"unit": "pf", "magnitude": 20}},
            }
        },
    }
]

experiment = Experiment(experiment_data)
experiment.run()

2024-04-09 13:46:47,692 - ............demos.enbios.base - INFO - Running scenario 'scenario1'


{'scenario1': {'name': 'root',
  'results': {'co2': {'unit': 'pf', 'magnitude': 20.0}},
  'output': [{'unit': 'liter', 'magnitude': 100.0}],
  'children': [{'name': 'simple_node',
    'results': {'co2': {'unit': 'pf', 'magnitude': 20.0}},
    'output': [{'unit': 'l', 'magnitude': 100.0}]}]}}

In [13]:
experiment.adapters[0].nodes

{'simple_node': SimpleAssignmentNodeConfig(node_name='simple_node', outputs=[SimpleAssignmentNodeOutput(unit='l', label=None)], default_outputs=[], default_impacts={'co2': ResultValue(unit='kg', magnitude=10.0, multi_magnitude=[])}, scenario_data={'scenario1': SimpleAssignmentNodeScenarioData(outputs=[NodeOutput(unit='l', magnitude=100.0, label=None)], impacts={'co2': ResultValue(unit='pf', magnitude=20.0, multi_magnitude=[])})})}

In [14]:
AssignmentAdapter.get_config_schemas()

{'node_name': {'$defs': {'NodeOutput': {'additionalProperties': False,
    'properties': {'unit': {'title': 'Unit', 'type': 'string'},
     'magnitude': {'default': 1.0, 'title': 'Magnitude', 'type': 'number'},
     'label': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
      'default': None,
      'title': 'Label'}},
    'required': ['unit'],
    'title': 'NodeOutput',
    'type': 'object'},
   'ResultValue': {'additionalProperties': False,
    'properties': {'unit': {'title': 'Unit', 'type': 'string'},
     'magnitude': {'anyOf': [{'type': 'number'}, {'type': 'null'}],
      'default': None,
      'title': 'Magnitude'},
     'multi_magnitude': {'anyOf': [{'items': {'type': 'number'},
        'type': 'array'},
       {'type': 'null'}],
      'title': 'Multi Magnitude'}},
    'required': ['unit'],
    'title': 'ResultValue',
    'type': 'object'},
   'SimpleAssignmentNodeOutput': {'properties': {'unit': {'title': 'Unit',
      'type': 'string'},
     'label': {'anyOf': [{'type': 's